In [5]:
#install faker library
!pip install faker

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.7 MB/s eta 0:00:00


In [137]:
#import libraries
from faker import Faker
import pandas as pd
import numpy as np
import random 
fake = Faker()

In [151]:
#generate synthetic data, future optimization is wrappping it all into a create_employee function. however this way is easier to test after each new variable addition.
employeeid =[ fake.unique.numerify(text = '########') for i in range(200)]
firstname = [fake.first_name() for i in range(200)]
lastname = [fake.last_name() for i in range(200)]
dob = [fake.date_of_birth() for i in range(200)]
ssn = [fake.unique.ssn() for i in range(200)]
hiredate = [fake.date_between(start_date = '-20y') for i in range(200)] #make the oldest employee 20 years of tenure
termdate = [fake.date_between(end_date = '+6m') for i in range(200)] #make policies expire in 6 months for new year
effectivedate = [fake.date_between(start_date = '-1y') for i in range(200)] #make policies start a year ago

In [153]:
#for the custom plan names
plan_list = [
'VIS-BUYUP','Dental – BASIC','PPO–Med Plan',
'Med_PPO','fsa-variation-1','Premium Dental Coverage',
'VISION Core Option','Medical HDHP','Basic Dental Plan',
'FSA_VOYA','vision-buy-up','Dental – Premium Plan',
'HDHP – Medical','Dental Premium' ,'PPO Medical','High Deductible Medical',
'Medical PPO Core','Dental Basic','MED PPO – Core']

planname = [fake.word(ext_word_list=plan_list) for i in range(200)]

In [163]:
coverage_list = ['Bronze','Gold','Silver','Platinum']
coveragelevel = [random.choice(coverage_list) for i in range(200) ]


In [245]:

df = pd.DataFrame({'employee_id':employeeid,'first_name':firstname,'last_name':lastname,'dob':dob,'ssn':ssn,'hire_date':hiredate,
                       'termdate':termdate,'plan_name':planname,'coverage_level':coveragelevel,'effective_date':effectivedate})


In [161]:
#save to csv
#df.to_csv('employees_inbound.csv')

In [255]:
#mapping plan names
manual_mapping = {'VIS-BUYUP':'Vision Premium', 
                  'Dental – BASIC':'Dental Basic' ,
                  'PPO–Med Plan':'Medical PPO',
                  'Med_PPO':'Medical PPO',
                  'fsa-variation-1':'Medical PPO',
                  'Premium Dental Coverage':'Dental Premium',
                  'VISION Core Option':'Vision Core',
                  'Medical HDHP':'Medical HDHP',
                  'Basic Dental Plan':'Dental Basic',
                  'FSA_VOYA':'Medical PPO', 
                  'vision-buy-up':'Vision Premium',
                  'Dental – Premium Plan':'Dental Premium',
                  'HDHP – Medical':'Medical HDHP',
                  'Dental Premium':'Dental Premium' ,
                 'PPO Medical':'Medical PPO',
                  'High Deductible Medical':'Medical HDHP',
                  'Medical PPO Core':'Medical PPO',
                  'Dental Basic':'Dental Basic',
                  'MED PPO – Core':'Medical PPO'}

df['original_plan_name'] = df['plan_name']  # save original
df['mapped_plan_name'] = df['plan_name'].map(manual_mapping)
#print(df['mapped_plan_name'].tail(50))
#check nan
#print("Unmapped entries:", df['mapped_plan_name'].isna().sum())
#
print(df['mapped_plan_name'].unique())

150    Dental Premium
151       Medical PPO
152       Medical PPO
153    Vision Premium
154    Dental Premium
155      Dental Basic
156       Medical PPO
157    Vision Premium
158       Medical PPO
159       Medical PPO
160       Medical PPO
161    Vision Premium
162    Dental Premium
163       Medical PPO
164    Dental Premium
165       Medical PPO
166    Dental Premium
167       Vision Core
168       Medical PPO
169       Medical PPO
170       Medical PPO
171      Medical HDHP
172      Medical HDHP
173      Medical HDHP
174       Medical PPO
175    Dental Premium
176       Medical PPO
177      Medical HDHP
178    Dental Premium
179       Medical PPO
180       Medical PPO
181      Dental Basic
182    Vision Premium
183      Medical HDHP
184      Medical HDHP
185    Dental Premium
186      Medical HDHP
187      Dental Basic
188      Dental Basic
189       Medical PPO
190    Dental Premium
191    Dental Premium
192       Medical PPO
193    Vision Premium
194       Medical PPO
195       

In [307]:
#ssn normalization , dates already in year-month-day format 
import re
def dash_remove(ssn):
    return re.sub('-','',ssn)
df['clean_ssn'] = df['ssn'].apply(dash_remove)
print(df['clean_ssn'].head())

0    415903602
1    230818934
2    714878829
3    255020827
4    087515952
Name: clean_ssn, dtype: object


In [311]:
#outputcsv
df['full_name']= df['first_name'] +' ' + df['last_name']
out = pd.DataFrame({'MemberID':df['employee_id'], 'Name':df['full_name'], 'DOB':df['dob'],'SSN': df['clean_ssn'], 
                    'Plan':df['mapped_plan_name'],'Coverage':df['coverage_level'], 'EffectiveDate':df['effective_date']
                   })
print(out.head())

   MemberID            Name         DOB        SSN            Plan  Coverage  \
0  66550557   Nicole Durham  2024-04-09  415903602     Medical PPO  Platinum   
1  79114177   Lindsay Velez  1989-10-19  230818934    Medical HDHP    Bronze   
2  33816824  Michael Farmer  1973-03-02  714878829     Medical PPO  Platinum   
3  68362109     Kevin Tapia  1922-01-18  255020827  Dental Premium    Bronze   
4  98141980     Kerri Lopez  1942-08-29  087515952  Dental Premium  Platinum   

  EffectiveDate  
0    2025-01-16  
1    2024-12-14  
2    2025-04-16  
3    2025-03-13  
4    2025-11-17  


In [313]:
#save csv
out.to_csv('carrier_export.csv')